# データで見る ゲンロン 大森望 SF創作講座 2018 付録

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
%config InlineBackend.figure_formats = {'png', 'retina'}
sns.set()

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%load_ext sql
%sql postgresql://fuji-nakahara@localhost/genron-school-sf-app_development

'Connected: fuji-nakahara@genron-school-sf-app_development'

In [4]:
term_id = 2018

In [5]:
from datetime import datetime
print(datetime.now())

2019-03-28 22:21:45.852955


## 文字数のレンジごとの梗概選出率比較

> 文字数のところ、梗概全体だと500文字以下などもありますし、1000～1999まで200文字ずつ切ったゾーンと2000～のゾーン、計6個のゾーンで該当梗概の選出割合を出してみるのもまたありかと思います。

In [6]:
%%sql synopses_character_counts <<

select
    s.number
    , sy.title
    , st.name as author
    , sy.character_count
    , sy.appeal_character_count
    , sy.selected
from
    subjects s
    join synopses sy on s.id = sy.subject_id
    join students st on sy.student_id = st.id
where
    s.term_id = :term_id
    and s.number <= 9
order by
    1

 * postgresql://fuji-nakahara@localhost/genron-school-sf-app_development
Returning data to local variable synopses_character_counts


In [7]:
selected_synopses_character_counts = synopses_character_counts[synopses_character_counts.selected]

In [8]:
bins = [0, 1000, 1200, 1400, 1600, 1800, 2000, synopses_character_counts.character_count.max()]

In [9]:
grouped_character_counts = pd.DataFrame({
    'synopses_count': synopses_character_counts \
        .character_count.groupby(pd.cut(synopses_character_counts.character_count, bins)).count(),
    'selected_synopses_count': selected_synopses_character_counts \
        .character_count.groupby(pd.cut(selected_synopses_character_counts.character_count, bins)).count()
})

In [10]:
grouped_character_counts['selected_rate'] = grouped_character_counts.selected_synopses_count / grouped_character_counts.synopses_count

In [11]:
grouped_character_counts

,synopses_count,selected_synopses_count,selected_rate
character_count,,,
"(0, 1000]",6,0,0.000000
"(1000, 1200]",98,10,0.102041
"(1200, 1400]",57,5,0.087719
"(1400, 1600]",35,7,0.200000
"(1600, 1800]",24,3,0.125000
"(1800, 2000]",17,3,0.176471
"(2000, 3714]",23,5,0.217391


なんと、もっとも梗概選出率が高いのは2000字よりも多いグループだった。  
ついで1400〜1600字のグループで、約2割の選出率。  
1000文字以下の選出率は0でもっとも低く、ついで1200〜1400字のグループ。  
1000〜1200字は1200〜1400時よりも高いので、少し字数を超えるくらいならがんばって1200字以下に抑えたほうがよいかもしれない。